<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/Rescrape_using_siddgood_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import numpy as np

import json
import re
import unidecode
import time
from tqdm import tqdm

from selenium import webdriver



In [ ]:
CATEGORIES = {
    'arts': "https://podcasts.apple.com/us/genre/podcasts-arts/id1301",
    'business': "https://podcasts.apple.com/us/genre/podcasts-business/id1321",
    'comedy': "https://podcasts.apple.com/us/genre/podcasts-comedy/id1303",
    'education': "https://podcasts.apple.com/us/genre/podcasts-education/id1304",
    'fiction': "https://podcasts.apple.com/us/genre/podcasts-fiction/id1483",
    'government': "https://podcasts.apple.com/us/genre/podcasts-government/id1511",
    'health': "https://podcasts.apple.com/us/genre/podcasts-health-fitness/id1512",
    'history': "https://podcasts.apple.com/us/genre/podcasts-history/id1487",
    'kids_and_family': "https://podcasts.apple.com/us/genre/podcasts-kids-family/id1305",
    'leisure': "https://podcasts.apple.com/us/genre/podcasts-leisure/id1502",
    'music': "https://podcasts.apple.com/us/genre/podcasts-music/id1310",
    'news': "https://podcasts.apple.com/us/genre/podcasts-news/id1489",
    'religion_and_spirituality': "https://podcasts.apple.com/us/genre/podcasts-religion-spirituality/id1314",
    'science': "https://podcasts.apple.com/us/genre/podcasts-science/id1533",
    'society_and_culture': "https://podcasts.apple.com/us/genre/podcasts-society-culture/id1324",
    'sports': "https://podcasts.apple.com/us/genre/podcasts-sports/id1545",
    'tv_and_film': "https://podcasts.apple.com/us/genre/podcasts-tv-film/id1309",
    'technology': "https://podcasts.apple.com/us/genre/podcasts-technology/id1318",
    'true_crime': "https://podcasts.apple.com/us/genre/podcasts-true-crime/id1488"
}

In [ ]:
all_podcast_links = list()

for category_url in CATEGORIES.values():
    response = requests.get(category_url, timeout=5)
    content = BeautifulSoup(response.content, "lxml")
    podcast_links = content.find('div', class_='grid3-column')

    for link in podcast_links.findAll('a'):
        all_podcast_links.append(link.get('href'))

all_podcast_links = list(set(all_podcast_links))

print('Number of podcast links', len(all_podcast_links))

with open('podcast_links.json', 'w') as outfile:
    json.dump(all_podcast_links, outfile)

Number of podcast links 4560


In [ ]:
def convert_si_to_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000  # convert K to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000  # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000  # convert B to a Billion
    else:
        total_stars = int(x)  # Less than 1000
    return int(total_stars)

In [ ]:

def build_podcast_object(link):
    try:
        response = requests.get(link, timeout=10)
        content = BeautifulSoup(response.content, "lxml")
        try:
            title = content.find('h1').find('span').text.strip()
        except AttributeError:
            title = 'NA'
        try:
            producer = content.find('h1').find('a').text.strip()
        except AttributeError:
            try:
                producer = content.find('h1').find('span', class_='product-header__identity podcast-header__identity') \
                    .text.strip()
            except AttributeError:
                producer = 'NA'
        try:
            genre = content.find('li', class_="product-header__list__item").text.strip()
        except AttributeError:
            genre = 'NA'
        try:
            description = content.find('div', class_="product-hero-desc product-hero-desc--side-bar").text.strip()
        except AttributeError:
            description = 'NA'
        try:
            num_episodes = int(content.find('div', class_="product-artwork__caption small-hide medium-show")
                               .text.strip().strip('episodes'))
        except AttributeError:
            num_episodes = 'NA'
        try:
            rating = float(content.find('span', class_="we-customer-ratings__averages__display").text.strip())
        except AttributeError:
            rating = 'NA'
        try:
            num_reviews = convert_si_to_number(content
                                               .find('div', class_="we-customer-ratings__count small-hide medium-show")
                                               .text.strip().strip('Ratings'))
        except AttributeError:
            num_reviews = 'NA'

        try:
            script_tag = content.find('script', {'name': 'schema:podcast-show', 'type': 'application/ld+json'})

            # Extract and load the JSON data
            json_data = json.loads(script_tag.string)

            # Access the features in the JSON data
            script = {'jname': json_data['name'], 
                      'jauthor': json_data['author'],
                      'jdescription': json_data['description'],        
                      'jdate_published': json_data['datePublished'],
                      'joffers': json_data['offers'],
                      'jreviews': json_data['review']}

            
        except AttributeError:
            script = 'NA'


    except Exception:
        title = 'NA'
        producer = 'NA'
        genre = 'NA'
        description = 'NA'
        num_episodes = 'NA'
        rating = 'NA'
        num_reviews = 'NA'
        script = 'NA'
    podcast_object = {
        'title': title,
        'producer': producer,
        'genre': genre,
        'description': script['jdescription'],
        'num_episodes': num_episodes,
        'rating': rating,
        'num_reviews': num_reviews,
        'link': link,
        'itunes_id': link.split('/')[-1].strip('id')
    }
    return podcast_object


In [ ]:
counter = 1
podcastObjects = list()
with open('podcast_links.json') as json_file:
    podcasts = json.load(json_file)
    for podcast_link in podcasts:
        try:
            podcast = build_podcast_object(podcast_link)
            podcastObjects.append(podcast)
            print('Completed Podcast: ' + str(counter))
            counter += 1
        except Exception:
            print('Failed on ' + podcast_link)
            counter += 1
            pass

with open('podcasts_info_anotherattempt.json', 'w') as outfile:
    json.dump(podcastObjects, outfile)

Completed Podcast: 1
Completed Podcast: 2
Completed Podcast: 3
Completed Podcast: 4
Completed Podcast: 5
Completed Podcast: 6
Completed Podcast: 7
Completed Podcast: 8
Completed Podcast: 9
Completed Podcast: 10
Completed Podcast: 11
Completed Podcast: 12
Completed Podcast: 13
Completed Podcast: 14
Completed Podcast: 15
Completed Podcast: 16
Completed Podcast: 17
Completed Podcast: 18
Completed Podcast: 19
Completed Podcast: 20
Completed Podcast: 21
Completed Podcast: 22
Completed Podcast: 23
Completed Podcast: 24
Completed Podcast: 25
Completed Podcast: 26
Completed Podcast: 27
Completed Podcast: 28
Completed Podcast: 29
Failed on https://podcasts.apple.com/us/podcast/the-npr-politics-podcast/id1057255460
Completed Podcast: 31
Completed Podcast: 32
Completed Podcast: 33
Completed Podcast: 34
Completed Podcast: 35
Completed Podcast: 36
Completed Podcast: 37
Completed Podcast: 38
Completed Podcast: 39
Completed Podcast: 40
Completed Podcast: 41
Completed Podcast: 42
Completed Podcast: 43

In [ ]:
sample_podcast = build_podcast_object(podcast_link)

In [ ]:
sample_podcast

{'title': 'TechTank',
 'producer': 'Brookings Institution',
 'genre': 'Technology',
 'description': 'TechTank is a biweekly podcast from The Brookings Institution exploring the most consequential technology issues of our time. From artificial intelligence and racial bias in algorithms, to Big Tech, the future of work, and the digital divide, TechTank takes abstract ideas and makes them accessible.…',
 'num_episodes': 66,
 'rating': 5.0,
 'num_reviews': 14,
 'link': 'https://podcasts.apple.com/us/podcast/techtank/id1526725061',
 'itunes_id': '1526725061'}

In [ ]:
def clean_title(t):
    t = unidecode.unidecode(t)
    t = t.replace('\n', ' ')
    t = re.sub(r'[^\w\s]', '', t)
    t = re.sub(r'\d+', '', t)
    t = t.lower()
    t = t.strip()
    return t


In [ ]:
def clean_description(d):
    d = unidecode.unidecode(d)
    d = d.replace('\n', ' ')
    d = re.sub(r'[^\w\s]', '', d)
    d = re.sub(r'\d+', '', d)
    if re.findall(r'(.*) brought to you by.*', d):
        d = re.sub(r'brought to you by.*', '', d)
    if re.search(r'(.*) sponsored by.*', d):
        d = re.sub(r'sponsored by.*', '', d)
    d = d.lower()
    d = d.strip()
    return d


In [ ]:
def get_recent_podcast_episodes(link):
    episode_titles = ''
    episode_desc = ''

    driver = webdriver.PhantomJS()
    driver.get(link)
    html = driver.page_source.encode('utf-8')

    soup = BeautifulSoup(html, 'lxml')
    text = str(soup.find('script'))

    try:
        text = text.split('"workExample":')[1].split(',"aggregateRating"')[0]
        episode_data = json.loads(text)

        for episode in episode_data:
            title = episode['name']
            c_title = clean_title(title)

            description = episode['description']
            c_description = clean_description(description)

            episode_titles += (c_title + " ")
            episode_desc += (c_description + " ")

        episode_titles = episode_titles.strip()
        episode_desc = episode_desc.strip()

    except Exception:
        episode_title = np.nan
        episode_desc = np.nan
        print("Failed on: " + str(link))

    return [episode_titles, episode_desc]

In [76]:
with open('podcasts_info_anotherattempt.json') as json_file:
        podcasts = json.load(json_file)

podcasts_df = pd.DataFrame(podcasts)

podcasts_df = podcasts_df[['title', 'producer', 'genre', 'description', 'num_episodes',
                               'rating', 'num_reviews', 'link', 'itunes_id']]
podcasts_df = podcasts_df.replace('NA', np.nan)
#podcasts_df = podcasts_df.dropna()

print(podcasts_df.head())

podcast_titles = list(podcasts_df['title'])
podcast_titles = [title.replace(" ", "") for title in podcast_titles]
podcast_titles = [re.sub(r'[^\w\s]', '', title) for title in podcast_titles]
is_english = [bool(re.match("^[A-Za-z9]*$", title)) for title in podcast_titles]
podcasts_df['is_english'] = is_english
podcasts_df = podcasts_df[podcasts_df.is_english == True]
podcasts_df = podcasts_df.drop(columns=['is_english'])

podcasts_df = podcasts_df.reset_index(drop=True)

podcasts_df = podcasts_df.reset_index(drop=True)

podcasts_df.to_pickle('re-scraped_podcasts.pkl')

                         title  \
0                 Kosmographia   
1      Coffee + Crumbs Podcast   
2                      Drilled   
3  Fantasy Baseball Today in 5   
4              Best Laid Plans   

                                            producer          genre  \
0                                    Randall Carlson        Science   
1          Katie Blackburn, Jill Atogwe, Ashlee Gadd  Kids & Family   
2                                 Critical Frequency        Science   
3  CBS Sports, Fantasy Baseball, MLB, Fantasy Ran...         Sports   
4                                   Sarah Hart-Unger      Education   

                                         description  num_episodes  rating  \
0                     All the Randall you can handle            95     4.9   
1  The Coffee + Crumbs Podcast is your companion ...           141     4.8   
2  A true-crime podcast about climate change, hos...           183     4.6   
3  Want Fantasy advice, news and analysis fast? F...    

In [77]:
loaded = pd.read_pickle('re-scraped_podcasts.pkl')
loaded

,title,producer,genre,description,num_episodes,rating,num_reviews,link,itunes_id
0,Kosmographia,Randall Carlson,Science,All the Randall you can handle,95,4.9,699.0,https://podcasts.apple.com/us/podcast/kosmogra...,1479346591
1,Coffee + Crumbs Podcast,"Katie Blackburn, Jill Atogwe, Ashlee Gadd",Kids & Family,The Coffee + Crumbs Podcast is your companion ...,141,4.8,1700.0,https://podcasts.apple.com/us/podcast/coffee-c...,1093725475
2,Drilled,Critical Frequency,Science,"A true-crime podcast about climate change, hos...",183,4.6,2100.0,https://podcasts.apple.com/us/podcast/drilled/...,1439735906
3,Best Laid Plans,Sarah Hart-Unger,Education,Welcome to Best Laid Plans -- a podcast all ab...,139,4.9,535.0,https://podcasts.apple.com/us/podcast/best-lai...,1525311647
4,The Sports Junkies,Audacy,Sports,"Cakes, Bish, EB and JP on 106.7 the Fan.",500,4.4,1500.0,https://podcasts.apple.com/us/podcast/the-spor...,367825854
...,...,...,...,...,...,...,...,...,...
4107,Oxventure - A Dungeons & Dragons Podcast,Oxventure,Leisure,The Oxventurers Guild presents a DnD real play...,134,5.0,655.0,https://podcasts.apple.com/us/podcast/oxventur...,1528568221
4108,Future of Infrastructure,Microsoft Podcasts,Technology,"Host Jeremy Goldberg, Worldwide Director of Cr...",15,NaN,NaN,https://podcasts.apple.com/us/podcast/future-o...,1674271676
4109,The TWIML AI Podcast (formerly This Week in Ma...,Sam Charrington,Technology,Machine learning and artificial intelligence a...,640,4.7,370.0,https://podcasts.apple.com/us/podcast/the-twim...,1116303051
4110,The Bestie Breakdown,The Bestie Breakdown,Kids & Family,Hi! We're Shay Shull from Mix and Match Mama a...,51,3.8,1400.0,https://podcasts.apple.com/us/podcast/the-best...,1509081920


In [79]:
get_recent_podcast_episodes(sample_podcast['link'])

AttributeError: ignored